# 3주차 실습 정리 
### car evaluation(자동차 평가) 분류하기 

파일불러와서 데이터를 확인

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix

# CSV 파일 읽기
file_path = "/content/drive/MyDrive/car_evaluation.csv"
df = pd.read_csv(file_path, header = None)

# 데이터 확인
print(df.head())
print(df.columns)

       0      1  2  3      4     5      6
0  vhigh  vhigh  2  2  small   low  unacc
1  vhigh  vhigh  2  2  small   med  unacc
2  vhigh  vhigh  2  2  small  high  unacc
3  vhigh  vhigh  2  2    med   low  unacc
4  vhigh  vhigh  2  2    med   med  unacc
Index([0, 1, 2, 3, 4, 5, 6], dtype='int64')


## 컬럼이 없음! 컬럼을 부여!

In [ ]:
import pandas as pd

df.columns = ["buying", "maint", "doors", "persons", "lug_capacity", "safety", "class"]
print(df.columns)
print(df.head())

Index(['buying', 'maint', 'doors', 'persons', 'lug_capacity', 'safety',
       'class'],
      dtype='object')
  buying  maint doors persons lug_capacity safety  class
0  vhigh  vhigh     2       2        small    low  unacc
1  vhigh  vhigh     2       2        small    med  unacc
2  vhigh  vhigh     2       2        small   high  unacc
3  vhigh  vhigh     2       2          med    low  unacc
4  vhigh  vhigh     2       2          med    med  unacc


## 컬럼이 생겼으니 다시 평소대로 분류시작

In [ ]:

# 결측치 확인
df.isnull().sum()

,0
buying,0
maint,0
doors,0
persons,0
lug_capacity,0
safety,0
class,0


In [ ]:
# 레이블 확인
print(df['class'].value_counts())

class
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64


## 숫자가 아닌 데이터들이 있으면 분류를 할 수 없기 때문에
## 데이터들을 모두 숫자로 교체

In [ ]:

from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for col in df.columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])  # 각 컬럼을 숫자로 변환
    label_encoders[col] = le  # 변환기 저장 (나중에 역변환 가능)

print(df.head())  # 변환된 데이터 확인

   buying  maint  doors  persons  lug_capacity  safety  class
0       3      3      0        0             2       1      2
1       3      3      0        0             2       2      2
2       3      3      0        0             2       0      2
3       3      3      0        0             1       1      2
4       3      3      0        0             1       2      2


## df.drop('class', axis=1).values
df.drop('class', axis=1) → class 열을 제외한 새로운 Pandas DataFrame 생성
.values → DataFrame을 **NumPy 배열(numpy.ndarray)** 로 변환
## df['class'].values
df['class'] → class 열을 Series 형태로 가져옴
.values → Series를 **NumPy 배열(numpy.ndarray)** 로 변환

In [ ]:
# 타겟 변수와 독립 변수 분리
X = df.drop('class', axis=1).values
y = df['class'].values

X


array([[3, 3, 0, 0, 2, 1],
       [3, 3, 0, 0, 2, 2],
       [3, 3, 0, 0, 2, 0],
       ...,
       [1, 1, 3, 2, 0, 1],
       [1, 1, 3, 2, 0, 2],
       [1, 1, 3, 2, 0, 0]])

In [ ]:
y

array([2, 2, 2, ..., 2, 1, 3])

In [ ]:
from sklearn.preprocessing import StandardScaler

#Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)


# 훈련용 데이터와 테스트용 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train.shape,X_test.shape, y_train.shape,y_test.shape

((1209, 6), (519, 6), (1209,), (519,))

In [ ]:

from sklearn.tree import DecisionTreeClassifier

# DT 모델 생성
dt_model = DecisionTreeClassifier(random_state=42)

# 모델 학습
dt_model.fit(X_train, y_train)

# 예측
y_pred = dt_model.predict(X_test)

# 정확도 확인
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# 혼동 행렬 확인
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

Accuracy: 97.11%
Confusion Matrix:
[[111   6   1   0]
 [  1  18   0   0]
 [  4   0 354   0]
 [  3   0   0  21]]


In [ ]:
from sklearn.linear_model import LogisticRegression

# LR모델 생성
lr_model = LogisticRegression(max_iter=1000, random_state=42)

# 모델 학습
lr_model.fit(X_train, y_train)

# 예측
y_pred_lr = lr_model.predict(X_test)

# 정확도 출력
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Accuracy: {accuracy_lr * 100:.2f}%")

# 혼동 행렬 계산 및 출력
cm_lr = confusion_matrix(y_test, y_pred_lr)
print("Confusion Matrix:")
print(cm_lr)

Accuracy: 66.28%
Confusion Matrix:
[[ 20   0  92   6]
 [  3   0  16   0]
 [ 34   0 323   1]
 [ 12   0  11   1]]


In [ ]:

from sklearn.svm import SVC

# SVM모델 생성 (커널은 RBF 사용)
svm_model = SVC(kernel='rbf', random_state=42)

# 모델 학습
svm_model.fit(X_train, y_train)

# 예측
y_pred_svm = svm_model.predict(X_test)

# 정확도 출력
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"Accuracy: {accuracy_svm * 100:.2f}%")

# 혼동 행렬 계산 및 출력
cm_svm = confusion_matrix(y_test, y_pred_svm)
print("Confusion Matrix:")
print(cm_svm)

Accuracy: 91.33%
Confusion Matrix:
[[104   1  12   1]
 [ 14   3   0   2]
 [ 12   0 346   0]
 [  3   0   0  21]]


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# KNN모델 생성
knn_model = KNeighborsClassifier(n_neighbors=5)

# 모델 학습
knn_model.fit(X_train, y_train)

# 예측
y_pred_knn = knn_model.predict(X_test)

# 정확도 출력
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"Accuracy: {accuracy_knn * 100:.2f}%")

# 혼동 행렬 계산 및 출력
cm_knn = confusion_matrix(y_test, y_pred_knn)
print("Confusion Matrix:")
print(cm_knn)

Accuracy: 92.49%
Confusion Matrix:
[[104   0  11   3]
 [ 11   7   0   1]
 [  4   0 354   0]
 [  6   1   2  15]]
